### 드론 tello 기초

In [2]:
from djitellopy import Tello
from time import sleep
import cv2
from threading import Thread

In [11]:
# 배터리 체크
def battery_check() : 
  drone = Tello()
  drone.connect()

  power = drone.get_battery()
  if power < 30 : print("배터리 부족", power)
  else : print("배터리", power)
  drone.end()
  return power

In [22]:
def check_cmd() :
  tello = Tello() # 드론 객체 생성
  tello.connect()
  power = tello.get_battery()
  
  while True :
    # 배터리 체크
    if power<30:
      print("배터리 부족")
      break
    
    # 배터리 출력
    print("power", power)
    # cmd 입력받아 실행
    cmd = input("명령어 입력 >>")

    if cmd == "exit" :
      print("프로그램 종료")
      break
    elif cmd == "takeoff" : 
      tello.takeoff()
    elif cmd == "land" : 
      tello.land()
    elif cmd == "forward" : 
      tello.move_forward(20) # cm 미터 단위
    elif cmd == "left" : 
      tello.move_left(20)
    elif cmd == "back" : 
      tello.move_back(30)
    elif cmd == "lclock" : 
      tello.rotate_clockwise(90)
    else : print("없는 명령어")
  tello.end()

check_cmd()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


43
없는 명령어
43
없는 명령어
43
프로그램 종료


### 미션 패드

In [20]:
# 미션패드
def mission_pads():
  tello = Tello()
  tello.connect()

  tello.enable_mission_pads()
  tello.set_mission_pad_detection_direction(0)
  # id 인식
  for i in range(5) :
    print(i)
    p_id = tello.get_mission_pad_id()
    print(p_id)
    if p_id == 1 : print("1이다! - 연우")
    sleep(1)

  tello.disable_mission_pads()

  tello.end()
mission_pads()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'mon'
[INFO] tello.py - 461 - Response mon: 'ok'
[INFO] tello.py - 437 - Send command: 'mdirection 0'
[INFO] tello.py - 461 - Response mdirection 0: 'ok'


0
-1
1
-1
2
-1
3
-1
4
-1


[INFO] tello.py - 437 - Send command: 'moff'
[INFO] tello.py - 461 - Response moff: 'ok'


### 영상처리 - 멀티 쓰레드

In [ ]:
# 동기 함수 
def work(name, count) :
  print("start work", name)
  for i in range(count) : 
    print(name, i)
    sleep(1)
  print("end work", name)

def main() :
  print("start main")
  """
  동기함수
  work("aaa", 10)
  work("bbb", 10)
  """
  # 부모 쓰레드
  thread = Thread(target=work, args=["aaa", 10])
  thread.start()
  work("bbb", 5)
  thread.join() # 부모가 끝날때까지 기다림

  print("end main")
main()

In [ ]:
def takeoff(tello) :
  tello.rotate_counter_clockwise(90)

def thread_takeoff() :
  tello = Tello()
  tello.connect()

  thread_takeoff = Thread(target=takeoff, args=[tello])
  thread_takeoff.start()

  for i in range(5) :
    print("name", i)
    sleep(1)
  
  thread_takeoff.join()
  print("착륙할거임")
  tello.land()
  tello.end()
thread_takeoff()

### 이미지 처리

In [4]:
def capture() : 
  tello = Tello()
  tello.connect()

  print(tello.get_battery())

  # 스트림은 앞쪽 카메라만 인식
  tello.streamon()
  sleep(3)

  while True :
    cmd = input("명령어 입력 >>")
    if cmd == "show" :
      img = tello.get_frame_read().frame
      print(type(img), img.shape)
      
      # 이미지 저장
      cv2.imwrite("stream_test.png", img)
      img = cv2.resize(img, (120,120))

      # 이미지 창으로 보기
      cv2.imshow("test", img)
      print(type(img), img.shape)
      cv2.waitKey(0)
      cv2.destroyAllWindows()
    elif cmd == "bye" :
      break
    else :
      print("명령어 없음")

  tello.streamoff()
  tello.end()
capture()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


27
<class 'numpy.ndarray'> (720, 960, 3)
<class 'numpy.ndarray'> (120, 120, 3)


[INFO] tello.py - 437 - Send command: 'streamoff'
[INFO] tello.py - 461 - Response streamoff: 'ok'


### 동영상 처리

In [6]:
def video():
  tello = Tello()
  tello.connect()
  
  power = tello.get_battery()
  print("power : ",power)
  
  # 스트림 실행
  tello.streamon()
  sleep(3)
  
  # 프레임 읽어오기
  frame_read = tello.get_frame_read()

  # 읽어온 프레임 계속 찍기
  while True:
    # frame으로 읽은 후 사이즈 변경 -> imshow()
    img = frame_read.frame
    img = cv2.resize(img,(120,120))
    cv2.imshow("video",img)
    # 키설정
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  
  cv2.destroyAllWindows()
  tello.streamoff()
  tello.end()
video()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[ERROR] tello.py - 457 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


power :  31


[INFO] tello.py - 437 - Send command: 'streamoff'
[INFO] tello.py - 461 - Response streamoff: 'ok'


### 노트북 카메라

In [5]:
def myCamera():
  cap = cv2.VideoCapture(0)
  
  while True :
    a, img = cap.read()
    # 읽기 못할 때
    if not a :
      print("Error")
      break;
    img = cv2.flip(img, 1)
    cv2.imshow("test", img)
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  cv2.waitKey(0)
  cv2.destroyAllWindows()

myCamera()

### 과제
- 실시간으로 영상을 받으면서 이륙할때 캡쳐하기

In [7]:
def video(tello):
  tello.connect()
  
  power = tello.get_battery()
  print("power : ",power)
  
  # 스트림 실행
  tello.streamon()
  sleep(3)
  
  # 프레임 읽어오기
  frame_read = tello.get_frame_read()

  # 읽어온 프레임 계속 찍기
  while True:
    # frame으로 읽은 후 사이즈 변경 -> imshow()
    img = frame_read.frame
    img = cv2.resize(img,(200,200))
    cv2.imshow("video",img)
    # 키설정
    key = cv2.waitKey(1)
    if key == ord('q'):
      break
  
  cv2.destroyAllWindows()
  tello.streamoff()
  tello.end()



def check_cmd(tello) :
  tello.connect()
  power = tello.get_battery()
  
  while True :
    # 배터리 체크
    if power<30:
      print("배터리 부족")
      break
    
    # 배터리 출력
    print("power", power)
    # cmd 입력받아 실행
    cmd = input("명령어 입력 >>")

    if cmd == "exit" :
      print("프로그램 종료")
      break
    elif cmd == "takeoff" : 
      tello.takeoff()
    elif cmd == "land" : 
      tello.land()
    elif cmd == "forward" : 
      tello.move_forward(20) # cm 미터 단위
    elif cmd == "left" : 
      tello.move_left(20)
    elif cmd == "back" : 
      tello.move_back(30)
    elif cmd == "lclock" : 
      tello.rotate_clockwise(90)
    else : print("없는 명령어")
  tello.end()

def main():
    tello = Tello()
    videoRun = Thread(target=video, args=[tello])
    
    flyRun = Thread(target=check_cmd, args=[tello])

    videoRun.start()
    sleep(10)
    flyRun.start()
    sleep(1)

    videoRun.join()
    flyRun.join()

    tello.land()
    sleep(3)
    tello.end()

main()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[ERROR] tello.py - 457 - 'utf-8' codec can't decode byte 0xcc in position 0: invalid continuation byte
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


power :  81


[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


power 79


[INFO] tello.py - 437 - Send command: 'takeoff'
[INFO] tello.py - 461 - Response takeoff: 'ok'


power 79


[INFO] tello.py - 437 - Send command: 'streamoff'
[INFO] tello.py - 461 - Response streamoff: 'ok'
[INFO] tello.py - 437 - Send command: 'land'
[INFO] tello.py - 461 - Response land: 'ok'


프로그램 종료


In [1]:
from djitellopy import Tello
from threading import Thread
import time
import cv2

def test01(drone):
    drone.connect()
    drone.streamon()
    time.sleep(3) #3초 대기를 주지 않으면 오류가 발생할 수 있음

    while True:
        img = drone.get_frame_read().frame
        img = cv2.resize(img, (120,120))
        cv2.imshow("120*120", img)
        Key = cv2.waitKey(1)
        if Key == ord("q"):
            break

    cv2.destroyAllWindows()
    drone.streamoff()
    drone.end()

# test01()

def test02(drone):
    drone.connect()
    power = drone.get_battery()

    if power<30:
        print("에너지가 부족함")
    else:
        while True:
            if power<30:
                print("에너지가 부족함")
                break
            print("이륙 착륙 앞쪽 뒷쪽 우측 좌측 시계 반시계")
            print("[5]  [.]  [8]  [2]  [6]  [4] [86] [42] ")
            cmd = input("명령어를 넣어주세요 >> ")
            
            if cmd=="end":
                print("프로그램을 종료하겠습니다.")
                break
            elif cmd=="5":
                drone.takeoff() #올라가기
            elif cmd==".":
                drone.land() #내려오기
            elif cmd=="8":
                drone.move_forward(20) #앞으로 이동
            elif cmd=="2":
                drone.move_back(20) #뒤로 이동
            elif cmd=="4":
                drone.move_left(20) #왼쪽으로 이동
            elif cmd=="6":
                drone.move_right(20) #오른쪽으로 이동
            elif cmd=="86":
                drone.rotate_clockwise(90) #시계방향
            elif cmd=="42":
                drone.rotate_counter_clockwise(90) #반시계방향
            else:
                print("알수없는 명령어!")
#test02()


def main():
    drone = Tello()
    videoRun = Thread(target=test01, args=[drone])
    
    flyRun = Thread(target=test02, args=[drone])

    videoRun.start()
    time.sleep(10)
    flyRun.start()
    time.sleep(1)

    videoRun.join()
    flyRun.join()

    drone.land()
    time.sleep(3)
    drone.end()

main()

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\nono\anaconda3\envs\test\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\nono\anaconda3\envs\test\lib\threading.py", line 870, in run
    self._target(*self._arg

KeyboardInterrupt: 

[WARNING] tello.py - 447 - Aborting command 'command'. Did not receive a response after 7 seconds
[INFO] tello.py - 437 - Send command: 'command'
